# Personal AI Agent with Quality Control

Build a personal AI agent that represents you on your website or portfolio.
This agent uses your LinkedIn profile and personal summary to answer questions
about your background, skills, and experience with built-in quality control.

## What You'll Learn
- Reading and processing PDF files (LinkedIn profiles)
- Building interactive UIs with Gradio
- Implementing quality control with LLM evaluators
- Creating self-correcting AI agents
- Using structured outputs with Pydantic

## Setup and Imports

In [ ]:
"""
Setup: Required Files and API Keys

1. Create a .env file with your API key:
   OPENAI_API_KEY=your_openai_key
   GOOGLE_API_KEY=your_google_key (optional, for evaluator)

2. Create a 'me' folder with:
   - linkedin.pdf: Your LinkedIn profile as PDF
   - summary.txt: A brief summary about yourself

⚠️ Remember to add .env to your .gitignore!
"""

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import os

In [ ]:
# Load environment variables
load_dotenv(override=True)
openai = OpenAI()

## Step 1: Load Your Personal Data

Replace the files in the 'me' folder with your own:
- Export your LinkedIn profile as PDF
- Create a summary.txt with key highlights

In [ ]:
# Read LinkedIn PDF
reader = PdfReader("me/linkedin.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print("LinkedIn profile loaded successfully!")
print(f"Total characters: {len(linkedin)}")

In [ ]:
# Display a sample of the LinkedIn content
print("Sample of LinkedIn content:")
print("=" * 50)
print(linkedin[:500] + "..." if len(linkedin) > 500 else linkedin)
print("=" * 50)

In [ ]:
# Read personal summary
try:
    with open("me/summary.txt", "r", encoding="utf-8") as f:
        summary = f.read()
    print("\nSummary loaded successfully!")
    print(f"Summary: {summary}")
except FileNotFoundError:
    print("⚠️ Warning: summary.txt not found. Please create one in the 'me' folder.")
    summary = "No summary available."

## Step 2: Configure Your Agent

Update this with your actual name!

In [ ]:
# Change this to your name!
name = "Your Name"

In [ ]:
# Create the system prompt for your personal agent
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
# Display the system prompt (first 500 characters)
print("System Prompt Preview:")
print("=" * 50)
print(system_prompt[:500] + "...")
print("=" * 50)

## Step 3: Basic Chat Function

Let's start with a simple chat interface before adding quality control.

In [ ]:
def chat_basic(message, history):
    """
    Basic chat function without quality control.
    
    Args:
        message: User's message
        history: Conversation history in Gradio format
    
    Returns:
        Agent's response
    """
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

### Test the Basic Agent

Uncomment the line below to launch the basic version:

In [ ]:
# Uncomment to launch basic version
# gr.ChatInterface(chat_basic, type="messages").launch()

## Step 4: Add Quality Control System

Now let's add an evaluator that checks the quality of responses
before sending them to the user.

In [ ]:
# Define structured output format for evaluation
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
# Create evaluator system prompt
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    """
    Creates the user prompt for the evaluator.
    
    Args:
        reply: The agent's response to evaluate
        message: The user's original message
        history: Conversation history
    
    Returns:
        Formatted prompt for evaluation
    """
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
# Set up evaluator (using Gemini, but you can use OpenAI too)
google_api_key = os.getenv("GOOGLE_API_KEY")

if google_api_key:
    gemini = OpenAI(
        api_key=google_api_key, 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    print("✓ Gemini evaluator configured")
else:
    print("○ Google API key not found - will use OpenAI for evaluation")
    gemini = openai  # Fall back to OpenAI

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    """
    Evaluates whether an agent's response is acceptable.
    
    Args:
        reply: The agent's response to evaluate
        message: The user's original message  
        history: Conversation history
    
    Returns:
        Evaluation object with is_acceptable (bool) and feedback (str)
    """
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)}
    ]
    
    try:
        # Try structured output (Gemini 2.0)
        response = gemini.beta.chat.completions.parse(
            model="gemini-2.0-flash-exp", 
            messages=messages, 
            response_format=Evaluation
        )
        return response.choices[0].message.parsed
    except:
        # Fallback for models without structured output
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        # Simple parsing
        content = response.choices[0].message.content.lower()
        is_acceptable = "acceptable" in content or "yes" in content
        return Evaluation(is_acceptable=is_acceptable, feedback=response.choices[0].message.content)

### Test the Evaluator

In [ ]:
# Test evaluation
test_messages = [{"role": "system", "content": system_prompt}]
test_response = "I have extensive experience in AI and machine learning."

test_evaluation = evaluate(test_response, "What's your background?", test_messages[:1])
print(f"Acceptable: {test_evaluation.is_acceptable}")
print(f"Feedback: {test_evaluation.feedback}")

## Step 5: Add Self-Correction

If the evaluator rejects a response, the agent will try again
with feedback about what went wrong.

In [ ]:
def rerun(reply, message, history, feedback):
    """
    Regenerates response with feedback from the evaluator.
    
    Args:
        reply: The rejected response
        message: User's original message
        history: Conversation history
        feedback: Evaluator's feedback on why it was rejected
    
    Returns:
        New improved response
    """
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    updated_system_prompt += "Please try again, addressing the feedback."
    
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Step 6: Complete Chat Function with Quality Control

In [ ]:
def chat(message, history):
    """
    Main chat function with quality control and self-correction.
    
    Process:
    1. Generate initial response
    2. Evaluate response quality
    3. If rejected, regenerate with feedback
    4. Return final response
    
    Args:
        message: User's message
        history: Conversation history
    
    Returns:
        Agent's final response (after quality control)
    """
    # Clean up history if needed (for non-OpenAI providers)
    # Uncomment if you get errors with Groq or other providers:
    # history = [{"role": h["role"], "content": h["content"]} for h in history]
    
    # Generate initial response
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content
    
    # Evaluate the response
    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("✓ Passed evaluation - returning reply")
        return reply
    else:
        print("✗ Failed evaluation - retrying")
        print(f"Feedback: {evaluation.feedback}")
        # Regenerate with feedback
        reply = rerun(reply, message, history, evaluation.feedback)
        print("✓ Generated improved response")
        return reply

## Step 7: Launch the Agent!

Run this cell to start your personal AI agent with quality control.

In [ ]:
# Launch the Gradio interface
gr.ChatInterface(
    chat, 
    type="messages",
    title=f"Chat with {name}'s AI Agent",
    description=f"Ask me questions about {name}'s background, skills, and experience!"
).launch()

## How It Works

### Architecture:
1. **User Question** → Agent generates response
2. **Evaluator** → Checks if response is acceptable
3. **Self-Correction** → If rejected, regenerates with feedback
4. **Final Response** → Sent to user

### Design Patterns Used:
- **Reflection Pattern**: Agent evaluates its own output
- **Self-Correction Pattern**: Agent improves based on feedback
- **Quality Control**: Automated evaluation before user sees response

### Key Features:
- ✅ Loads personal data from PDF and text files
- ✅ Interactive chat interface with Gradio
- ✅ Automated quality control
- ✅ Self-correcting responses
- ✅ Structured outputs with Pydantic
- ✅ Professional representation

## Extension Ideas

1. **Multiple Evaluators**: Use different models to vote on quality
2. **Evaluation Criteria**: Add specific checks (tone, accuracy, length)
3. **Learning System**: Track which responses get rejected and why
4. **Multi-Language**: Support questions in different languages
5. **Voice Interface**: Add speech-to-text and text-to-speech
6. **Analytics**: Track common questions and improve responses
7. **A/B Testing**: Compare different system prompts
8. **Context Memory**: Remember previous conversations
9. **Source Citations**: Reference specific parts of LinkedIn/summary
10. **Personality Tuning**: Adjust tone based on user feedback

## Tips for Better Results

### LinkedIn PDF:
- Export from LinkedIn: Profile → More → Save to PDF
- Ensure text is selectable (not an image)
- Include all relevant sections

### Summary.txt:
- Highlight unique achievements
- Include specific technologies/skills
- Mention notable projects
- Keep it concise (200-500 words)

### System Prompt:
- Be specific about tone (professional, friendly, technical)
- Include examples of good responses
- Set clear boundaries (what not to discuss)
- Customize for your audience

### Evaluator:
- Define specific quality criteria
- Include examples of good/bad responses
- Adjust strictness based on use case
- Consider multiple evaluation rounds

## Troubleshooting

### "FileNotFoundError: me/linkedin.pdf"
- Create a 'me' folder in the same directory
- Add your linkedin.pdf file
- Or update the path in the code

### "Gradio connection error"
- Check if port 7860 is available
- Try: `gr.ChatInterface(...).launch(server_port=7861)`

### "Evaluator always accepts/rejects"
- Review evaluator_system_prompt
- Add more specific criteria
- Test with different questions

### "History format error" (Groq, etc.)
- Uncomment the history cleaning line in chat()
- Some providers are strict about message format

## Notes

- **Privacy**: All processing happens via API calls
- **Costs**: Uses gpt-4o-mini (very affordable)
- **Performance**: Usually responds in 2-5 seconds
- **Quality**: Self-correction significantly improves responses
- **Customization**: Easily adaptable for different use cases